In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
nasdaq_ticker = "^IXIC"
nse_ticker = "^NSEI"

# Correlation Analysis:
**a) Collect historical data for NASDAQ and NSE indices.**

In [3]:
# Download historical data for indices
nasdaq_data = yf.download(nasdaq_ticker, start='2010-01-01', end='2023-05-01')
nse_data = yf.download(nse_ticker, start='2010-01-01', end='2023-05-01')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


**b) Calculate the correlation coefficient between the two indices.**

In [4]:
# Extracting the 'Close' prices from the historical data
nasdaq_close = nasdaq_data['Close']
nse_close = nse_data['Close']

# Calculate the correlation coefficient
correlation = nasdaq_close.corr(nse_close)
print("Correlation Coefficient:", correlation)


Correlation Coefficient: 0.9513138758576779


**c) Analyze the strength and direction of the relationship.**

The correlation coefficient is close to 1, so it indicates a strong positive relationship, meaning that when one index goes up, the other tends to go up as well.

# Lead-Lag Relationship:
**a) Identify potential lead-lag relationships between the indices.**

In [5]:
# Define the lagged time periods to analyze
lag_periods = range(1, 21)  # Analyze lag periods from 1 to 20 days

# Calculate the lagged correlation coefficients
lagged_correlations = []
for lag in lag_periods:
    nasdaq_lagged = nasdaq_close.shift(lag)
    correlation = nasdaq_lagged.corr(nse_close)
    lagged_correlations.append(correlation)

# Find the lag period with the highest correlation coefficient
max_correlation = np.max(lagged_correlations)
max_correlation_lag = lag_periods[np.argmax(lagged_correlations)]
print("Maximum Correlation:", max_correlation)
print("Lag Period with Maximum Correlation:", max_correlation_lag)

Maximum Correlation: 0.9529138735366324
Lag Period with Maximum Correlation: 20


**b) Analyze data to determine consistent leading or lagging behavior.**

As lag period with maximum correlation is positive and equal to 20, so NASDAQ leads NSE by 20 number of days

**c) Determine the index to be used for parameter optimization.**

As NASDAQ leads NSE, so we will use NASDAQ for parameter optimization.

**d) Explanation for choosing the index for parameter optimization**

This is due to the fact that leading index tends to provide better parameter optimization results.

# Indicator Coding:
**a) Code Keltner Channel, Bollinger Bands, and MACD indicators.**

In [6]:
# Defining function for keltner channel
def calculate_keltner_channel(high, low, close, n=20):
    typical_price = (high + low + close) / 3
    atr = typical_price.diff().abs().rolling(n).mean()
    keltner_middle = typical_price.rolling(n).mean()
    keltner_upper = keltner_middle + (2 * atr)
    keltner_lower = keltner_middle - (2 * atr)
    return keltner_middle, keltner_upper, keltner_lower

# Defining function for bollinger bands
def calculate_bollinger_bands(data, window_size, num_std):
    rolling_mean = data.rolling(window=window_size).mean()
    rolling_std = data.rolling(window=window_size).std()
    
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    
    return rolling_mean, upper_band, lower_band

# Defining function for MACD
def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    
    ema_short = data.ewm(span=short_period, adjust=False).mean()
    ema_long = data.ewm(span=long_period, adjust=False).mean()
    macd_line = ema_short - ema_long
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    macd_histogram = macd_line - signal_line
    
    return macd_line, signal_line, macd_histogram
# Calculation of Keltner Channel
nasdaq_data['keltner_middle'], nasdaq_data['keltner_upper'], nasdaq_data['keltner_lower'] = calculate_keltner_channel(
    nasdaq_data['High'], nasdaq_data['Low'], nasdaq_data['Close'], n=20
)

# Calculation of Bollinger Bands
nasdaq_data['bb_middle'], nasdaq_data['bb_upper'], nasdaq_data['bb_lower'] = calculate_bollinger_bands(nasdaq_data['Close'], 20, 2 
)

# Calculating MACD
nasdaq_data['macd'], nasdaq_data['macd_signal'], nasdaq_data['macd_hist'] = calculate_macd(
    nasdaq_data['Close']
)

In [7]:
def calculate_volatility(data, window=252):
    returns = np.log(data['Close'] / data['Close'].shift(1))
    volatility = returns.rolling(window).std() * np.sqrt(252)
    
    return volatility

def calculate_max_drawdown(data):
    cumulative_returns = (1 + data['Returns']).cumprod()
    max_drawdown = 1 - cumulative_returns.div(cumulative_returns.cummax())
    
    return max_drawdown

def calculate_sharpe_ratio(returns, risk_free_rate=0):
    excess_returns = returns - risk_free_rate
    sharpe_ratio = excess_returns.mean() / excess_returns.std()
    
    return sharpe_ratio

def calculate_sortino_ratio(returns, risk_free_rate=0):
    downside_returns = returns[returns < risk_free_rate]
    sortino_ratio = (returns.mean() - risk_free_rate) / downside_returns.std()
    
    return sortino_ratio

# Parameter Optimization:

**a) Optimize parameters for the indicators on NASDAQ.</br>
b) Used metrics coded in last assignment(volatality, max dropdown, sharpe ratio, sortino ratio) to evaluate strategy.</br>
c) Documented the optimized parameters for future reference.**

In [13]:
def optimize_parameters(data, indicator):
    best_params = {'window': None, 'std_dev': None,
                   'volatility': float('inf'), 'max_drawdown': float('inf'),
                   'sharpe_ratio': float('-inf'), 'sortino_ratio': float('-inf')}
    
    if indicator == 'keltner':
        for window in range(10, 50, 5):
            for multiplier in range(1, 4):
                middle, upper_band, lower_band = calculate_keltner_channel(data, window, multiplier)
                returns = (data['Close'] / data['Close'].shift(1)) - 1
                volatility = returns.std() * np.sqrt(252)
                sharpe_ratio = (returns.mean() - 0.02) / volatility
                downside_returns = returns.copy()
                downside_returns[returns >= 0] = 0
                sortino_ratio = (returns.mean() - 0.02) / np.sqrt((downside_returns**2).mean())    
                max_drawdown = (data['Close'].rolling(window, min_periods=1).max() - data['Close']) / data['Close'].rolling(window, min_periods=1).max()
                max_drawdown = max_drawdown.max()
                
                if (volatility < best_params['volatility']).all() and \
                   (max_drawdown < best_params['max_drawdown']) and \
                   (sharpe_ratio > best_params['sharpe_ratio']).all() and \
                   (sortino_ratio > best_params['sortino_ratio']).all():
                    
                    best_params['window'] = window
                    best_params['multiplier'] = multiplier
                    best_params['volatility'] = volatility
                    best_params['max_drawdown'] = max_drawdown
                    best_params['sharpe_ratio'] = sharpe_ratio
                    best_params['sortino_ratio'] = sortino_ratio
                    
    elif indicator == 'bollinger':
        for window in range(10, 50, 5):
            for std_dev in [1, 1.5, 2]:
                rolling_mean, upper_band, lower_band = calculate_bollinger_bands(data, window, std_dev)
                returns = (data['Close'] / data['Close'].shift(1)) - 1
                volatility = returns.std() * np.sqrt(252)
                sharpe_ratio = (returns.mean() - 0.02) / volatility
                downside_returns = returns.copy()
                downside_returns[returns >= 0] = 0
                sortino_ratio = (returns.mean() - 0.02) / np.sqrt((downside_returns**2).mean())    
                max_drawdown = (data['Close'].rolling(window, min_periods=1).max() - data['Close']) / data['Close'].rolling(window, min_periods=1).max()
                max_drawdown = max_drawdown.max()
                
                if (volatility < best_params['volatility']).all() and \
                   (max_drawdown < best_params['max_drawdown']) and \
                   (sharpe_ratio > best_params['sharpe_ratio']).all() and \
                   (sortino_ratio > best_params['sortino_ratio']).all():
                    
                    best_params['window'] = window
                    best_params['std_dev'] = std_dev
                    best_params['volatility'] = volatility
                    best_params['max_drawdown'] = max_drawdown
                    best_params['sharpe_ratio'] = sharpe_ratio
                    best_params['sortino_ratio'] = sortino_ratio
                    
    elif indicator == 'macd':
        for short_window in range(5, 20, 5):
            for long_window in range(20, 50, 5):
                for signal_window in range(5, 20, 5):
                    if signal_window < short_window or signal_window < long_window:
                        continue
                    macd_line, signal_line, histogram = calculate_macd(data, short_window, long_window, signal_window)
                    # Calculate trading signals and evaluate performance
                    data['Signal'] = np.where(macd_line > signal_line, 1, -1)
                    data['Strategy Returns'] = data['Signal'].shift(1) * data['Returns']
                    
                    # Calculate metrics
                    volatility = calculate_volatility(data)
                    max_drawdown = calculate_max_drawdown(data)
                    sharpe_ratio = calculate_sharpe_ratio(data['Strategy Returns'])
                    sortino_ratio = calculate_sortino_ratio(data['Strategy Returns'])
                    if (volatility < best_params['volatility']).all() and \
                       (max_drawdown < best_params['max_drawdown']).all() and \
                       (sharpe_ratio > best_params['sharpe_ratio']).all() and \
                       (sortino_ratio > best_params['sortino_ratio']).all():
                        
                        best_params['short_window'] = short_window
                        best_params['long_window'] = long_window
                        best_params['signal_window'] = signal_window
                        best_params['volatility'] = volatility
                        best_params['max_drawdown'] = max_drawdown
                        best_params['sharpe_ratio'] = sharpe_ratio
                        best_params['sortino_ratio'] = sortino_ratio
            
    return best_params


In [14]:

# Parameter optimization for Keltner Channel
nasdaq_best_keltner_params = optimize_parameters(nasdaq_data, 'keltner')
print("Optimized Keltner Channel Parameters for NASDAQ:")
print(nasdaq_best_keltner_params)

# Parameter optimization for Bollinger Bands
nasdaq_best_bollinger_params = optimize_parameters(nasdaq_data, 'bollinger')
print("\nOptimized Bollinger Bands Parameters for NASDAQ:")
print(nasdaq_best_bollinger_params)

# Parameter optimization for MACD
nasdaq_best_macd_params = optimize_parameters(nasdaq_data, 'macd')
print("\nOptimized MACD Parameters for NASDAQ:")
print(nasdaq_best_macd_params)

Optimized Keltner Channel Parameters for NASDAQ:
{'window': 10, 'std_dev': None, 'volatility': 0.20605521017000292, 'max_drawdown': 0.2343622692409484, 'sharpe_ratio': -0.09423687803436376, 'sortino_ratio': -2.089914729300807, 'multiplier': 1}

Optimized Bollinger Bands Parameters for NASDAQ:
{'window': 10, 'std_dev': 1, 'volatility': 0.20605521017000292, 'max_drawdown': 0.2343622692409484, 'sharpe_ratio': -0.09423687803436376, 'sortino_ratio': -2.089914729300807}

Optimized MACD Parameters for NASDAQ:
{'window': None, 'std_dev': None, 'volatility': inf, 'max_drawdown': inf, 'sharpe_ratio': -inf, 'sortino_ratio': -inf}


# Signal Generation:
**a) Apply optimized parameters to the other index.**

In [20]:
nse_data['KC_Middle'] = nse_data['Close'].rolling(nasdaq_best_keltner_params['window']).mean()
nse_data['KC_ATR'] = nse_data['High'] - nse_data['Low']
nse_data['KC_Upper'] = nse_data['KC_Middle'] + (nasdaq_best_keltner_params['multiplier'] * nse_data['KC_ATR'].rolling(nasdaq_best_keltner_params['window']).mean())
nse_data['KC_Lower'] = nse_data['KC_Middle'] - (nasdaq_best_keltner_params['multiplier'] * nse_data['KC_ATR'].rolling(nasdaq_best_keltner_params['window']).mean())

signals = []
position = 0

for i in range(len(nse_data)):
    if nse_data['Close'][i] > nse_data['KC_Upper'][i]:
        if position == 0:
            signals.append(1)  # Buy signal
            position = 1
        elif position == -1:
            signals.append(1)  # Buy signal to close short position
            position = 0
        else:
            signals.append(0)  # No signal
    elif nse_data['Close'][i] < nse_data['KC_Lower'][i]:
        if position == 0:
            signals.append(-1)  # Sell signal to open short position
            position = -1
        elif position == 1:
            signals.append(-1)  # Sell signal
            position = 0
        else:
            signals.append(0)  # No signal
    else:
        signals.append(0)  # No signal

# Create a DataFrame with dates and signals
signal_dates = nse_data.index
signals_df = pd.DataFrame({'Date': signal_dates, 'Keltner Channel Signals': signals})

# Print the signals and their respective dates
print(signals_df)

           Date  Keltner Channel Signals
0    2010-01-04                        0
1    2010-01-05                        0
2    2010-01-06                        0
3    2010-01-07                        0
4    2010-01-08                        0
...         ...                      ...
3263 2023-04-24                        0
3264 2023-04-25                        0
3265 2023-04-26                        0
3266 2023-04-27                        0
3267 2023-04-28                        0

[3268 rows x 2 columns]
